In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import zipfile

# with zipfile.ZipFile('/content/drive/MyDrive/pest-20250508T182454Z-001.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/MyDrive/pest')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras import regularizers
import os
import time

In [ ]:
base_dir = '/content/drive/MyDrive/pest/pest'


train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
#No of Classes
train_classes = sorted(os.listdir(train_dir))
test_classes = sorted(os.listdir(test_dir))

if not train_classes:
    print("No class directories found in the train folder")
else:
    print(f"Found {len(train_classes)} classes: {train_classes}")

In [ ]:
from PIL import Image, UnidentifiedImageError
import os

def clean_corrupted_images(directory):
    print(f"\nChecking: {directory}")
    deleted_count = 0

    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)

        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    with Image.open(img_path) as img:
                        img.verify()
                except (IOError, UnidentifiedImageError) as e:
                    print(f"Deleting corrupted image: {img_path} - Reason: {e}")
                    os.remove(img_path)
                    deleted_count += 1

    if deleted_count == 0:
        print("No corrupted images found.")
    else:
        print(f"Done. Deleted {deleted_count} corrupted image(s).")

# Run cleaning
clean_corrupted_images(train_dir)
clean_corrupted_images(test_dir)

In [ ]:
import os

def count_images_per_class(base_dir, class_names):
    image_counts = {}
    total = 0
    for cls in class_names:
        class_path = os.path.join(base_dir, cls)
        num_images = len([
            f for f in os.listdir(class_path)
            if os.path.isfile(os.path.join(class_path, f))
        ])
        image_counts[cls] = num_images
        total += num_images
    return image_counts, total

# Get class names
train_classes = sorted(os.listdir(train_dir))
test_classes = sorted(os.listdir(test_dir))

# Count images
train_counts, total_train = count_images_per_class(train_dir, train_classes)
test_counts, total_test = count_images_per_class(test_dir, test_classes)

# Collect all unique classes (for safety, in case train/test differ)
all_classes = sorted(set(train_counts.keys()) | set(test_counts.keys()))

# Header
print("Images per Class")
print("=" * 59)
print(f"{'Class Name':<30}{'Train Images':>15}{'Test Images':>15}")
print("=" * 59)

# Rows
for cls in all_classes:
    train_num = train_counts.get(cls, 0)
    test_num = test_counts.get(cls, 0)
    print(f"{cls:<30}{train_num:>15}{test_num:>15}")

# Totals
print("=" * 59)
print(f"{'Total training images:':<30}{total_train}")
print(f"{'Total testing images:':<30}{total_test}")
print(f"{'Total images:':<30}{total_train + total_test}")

In [ ]:
# Define image size and batch size
IMG_HEIGHT, IMG_WIDTH = 150, 150
IMG_CHANNELS = 3
BATCH_SIZE = 32

In [ ]:
# Enhanced data augmentation to reduce overfitting
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Increased from 15
    width_shift_range=0.2,  # Increased from 0.1
    height_shift_range=0.2,  # Increased from 0.1
    horizontal_flip=True,
    zoom_range=0.2,  # Added
    shear_range=0.2,  # Added
    validation_split=0.2
)

In [ ]:
train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

In [ ]:
val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

In [ ]:
test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

In [ ]:
# Visualization-only generator (no rescale)
viz_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

sample_img, _ = next(train_gen)  # Get a batch
img = sample_img[0]

# Convert sample image back to uint8 for visualization
img_uint8 = (img * 255).astype(np.uint8)  # Assuming img was normalized

# Generate augmented images
augmented_images = [img_uint8]
for _ in range(8):
    aug_iter = viz_datagen.flow(np.expand_dims(img_uint8, axis=0), batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    augmented_images.append(aug_img)

plt.figure(figsize=(10, 10))
for i, aug_img in enumerate(augmented_images):
    plt.subplot(3, 3, i + 1)
    plt.imshow(aug_img)
    plt.axis('off')
    plt.title("Original" if i == 0 else f"Augmented {i}")
plt.suptitle("Augmentation Variations")
plt.tight_layout()
plt.show()


In [ ]:
# Get class names and number of classes
class_names = list(train_gen.class_indices.keys())
NUM_CLASSES = len(class_names)
print(f"Number of classes: {NUM_CLASSES}")
print(f"Class names: {class_names}")

In [ ]:
# Compute class weights for imbalanced classes
labels = train_gen.classes
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

In [ ]:
# Show class distribution
unique, counts = np.unique(labels, return_counts=True)
print("Class distribution:")
for cls, count in zip(unique, counts):
    print(f"Class {class_names[int(cls)]}: {count} samples")

In [ ]:
# Plot class distribution
plt.figure(figsize=(6, 4))
plt.bar(class_names, counts)
plt.xticks(rotation=45)
plt.title("Class Distribution")
plt.ylabel("Number of images")
plt.show()

In [ ]:
# Display sample images
x_batch, y_batch = next(train_gen)
plt.figure(figsize=(6, 6))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_batch[i])
    plt.title(class_names[int(y_batch[i])])
    plt.axis('off')
plt.suptitle("Sample Images")
plt.show()

In [ ]:
# Define callbacks for underfitting
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),  # Stop if val loss doesn't improve
    ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)  # Reduce LR on plateau
]

In [ ]:
# Build baseline CNN with Dropout and L2 to reduce overfitting
model_baseline = Sequential([
    Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Compile model
model_baseline.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_baseline.summary()

In [ ]:
# Train model
history_baseline = model_baseline.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,  # Increased to address underfitting
    class_weight=class_weights_dict,  # Handle class imbalance
    callbacks=callbacks
)

In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_baseline.history['accuracy'], label='Train Acc')
plt.plot(history_baseline.history['val_accuracy'], label='Val Acc')
plt.title("Baseline: Accuracy")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history_baseline.history['loss'], label='Train Loss')
plt.plot(history_baseline.history['val_loss'], label='Val Loss')
plt.title("Baseline: Loss")
plt.legend()
plt.show()

In [ ]:
# Evaluate on test set
test_loss, test_acc = model_baseline.evaluate(test_gen, verbose=0)
print(f"Baseline Test Accuracy: {test_acc:.4f}")

In [ ]:
# Predictions and classification report
y_pred_baseline = model_baseline.predict(test_gen)
y_pred_classes = np.argmax(y_pred_baseline, axis=1)
y_true = test_gen.classes
print("Classification Report (Baseline):")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title("Confusion Matrix (Baseline)")
plt.show()

In [ ]:
# Sample predictions
x_test_batch, y_test_batch = next(test_gen)
y_pred_batch = np.argmax(model_baseline.predict(x_test_batch), axis=1)

# Randomly shuffle indices within the batch to display different images each time
random_indices = np.random.permutation(x_test_batch.shape[0])[:9]  # Select 9 random indices from 0 to 31

plt.figure(figsize=(10, 10))  # Larger figure for clarity
for i, idx in enumerate(random_indices):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test_batch[idx])

    pred_label = class_names[y_pred_batch[idx]]
    true_label = class_names[int(y_test_batch[idx])]

    # Use contrasting text with bounding box for better visibility
    plt.title(f"P: {pred_label}\nT: {true_label}", fontsize=10,
              color='white', backgroundcolor='black', loc='left')
    plt.axis('off')

plt.suptitle("Sample Predictions (Baseline)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit suptitle
plt.show()

In [ ]:
# Function to build deeper CNN with tuned regularization
def build_deeper_cnn():
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        # Block 2
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Block 3
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        # Block 4 (New)
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.35),

        # Classifier
        Flatten(),
        Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

In [ ]:
# Build and compile deeper model
model_deeper = build_deeper_cnn()
model_deeper.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_deeper.summary()

In [ ]:
epochs = 50
# Train deeper model
start = time.time()
history_deeper = model_deeper.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weights_dict,
    callbacks=callbacks
)
print(f"Deeper model training time: {time.time() - start:.1f} seconds")

In [ ]:
# Plot accuracy/loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_deeper.history['accuracy'], label='Train Acc')
plt.plot(history_deeper.history['val_accuracy'], label='Val Acc')
plt.title("Deeper Model Accuracy")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history_deeper.history['loss'], label='Train Loss')
plt.plot(history_deeper.history['val_loss'], label='Val Loss')
plt.title("Deeper Model Loss")
plt.legend()
plt.show()

In [ ]:
# Evaluate deeper model
test_loss_d, test_acc_d = model_deeper.evaluate(test_gen, verbose=0)
print(f"Deeper Model Test Accuracy: {test_acc_d:.4f}")

In [ ]:
# Predictions
y_pred_deeper = np.argmax(model_deeper.predict(test_gen), axis=1)
print("Classification Report (Deeper):")
print(classification_report(y_true, y_pred_deeper, target_names=class_names))
cm_d = confusion_matrix(y_true, y_pred_deeper)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_d, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title("Confusion Matrix (Deeper)")
plt.show()

In [ ]:
# Sample predictions (Deeper)
y_pred_batch = np.argmax(model_deeper.predict(x_test_batch), axis=1)

# Randomly shuffle indices within the batch to display different images each time
random_indices = np.random.permutation(x_test_batch.shape[0])[:9]  # Select 9 random indices from 0 to 31

plt.figure(figsize=(10, 10))  # Bigger size for better visibility
for i, idx in enumerate(random_indices):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test_batch[idx])

    pred_label = class_names[y_pred_batch[idx]]
    true_label = class_names[int(y_test_batch[idx])]

    # Use white text on black background for clarity
    plt.title(f"P: {pred_label}\nT: {true_label}", fontsize=10,
              color='white', backgroundcolor='black', loc='left')
    plt.axis('off')

plt.suptitle("Sample Predictions (Deeper)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
from tensorflow.keras.optimizers import SGD

# Compare optimizers
adam_acc = test_acc_d

# Deeper model with SGD
model_deeper_sgd = build_deeper_cnn()
model_deeper_sgd.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
start = time.time()
history_sgd = model_deeper_sgd.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    class_weight=class_weights_dict,
    callbacks=callbacks
)
sgd_time = time.time() - start
_, sgd_acc = model_deeper_sgd.evaluate(test_gen, verbose=0)
print(f"SGD Test Accuracy: {sgd_acc:.4f} (Time: {sgd_time:.1f}s)")

In [ ]:
# Report comparison
print(f"Baseline Test Accuracy: {test_acc:.4f}")
print(f"Deeper (Adam) Test Accuracy: {adam_acc:.4f}")
print(f"Deeper (SGD) Test Accuracy: {sgd_acc:.4f}")

In [ ]:
# Summary table
import pandas as pd
results = pd.DataFrame({
    'Model': ['Baseline', 'Deeper (Adam)', 'Deeper (SGD)'],
    'Test Accuracy': [test_acc, adam_acc, sgd_acc]
})
print("\nSummary of Results:")
print(results)

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
# Image augmentation and preprocessing for MobileNetV1
mobilenet_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input  # MobileNetV1 specific preprocessing
)

In [ ]:
# Train, validation, and test data generators
train_gen_mobilenet = mobilenet_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

In [ ]:
val_gen_mobilenet = mobilenet_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)


In [ ]:
test_gen_mobilenet = mobilenet_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV1

def build_mobilenetv1_frozen():
    # Load MobileNetV1 without the top layer and with pretrained ImageNet weights
    base_model = MobileNetV1(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    
    # Freeze the base model layers (feature extraction)
    base_model.trainable = False
    
    # Build custom classifier
    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    x = base_model(inputs, training=False)  # Freeze batch normalization layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model

In [ ]:
# Build the frozen model
mobilenet_frozen_model = build_mobilenetv1_frozen()
mobilenet_frozen_model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

mobilenet_frozen_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

mobilenet_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('mobilenet_v1_frozen_best.keras', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]


In [ ]:
mobilenet_epochs = 50
import time

start_time = time.time()

history_mobilenet_frozen = mobilenet_frozen_model.fit(
    train_gen_mobilenet,
    validation_data=val_gen_mobilenet,
    epochs=mobilenet_epochs,
    callbacks=mobilenet_callbacks
)

mobilenet_frozen_time = time.time() - start_time


In [ ]:
test_loss_mobilenet_frozen, test_acc_mobilenet_frozen = mobilenet_frozen_model.evaluate(test_gen_mobilenet, verbose=0)
print(f"MobileNetV1 (Frozen) Test Accuracy: {test_acc_mobilenet_frozen:.4f}")

# Get predictions
y_pred_mobilenet_frozen = np.argmax(mobilenet_frozen_model.predict(test_gen_mobilenet), axis=1)
y_true_mobilenet = test_gen_mobilenet.classes


In [ ]:
from sklearn.metrics import classification_report

print("Classification Report (MobileNetV1 Frozen):")
print(classification_report(y_true_mobilenet, y_pred_mobilenet_frozen, target_names=class_names))


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_mobilenet_frozen.history['accuracy'], label='Train Acc')
plt.plot(history_mobilenet_frozen.history['val_accuracy'], label='Val Acc')
plt.title("MobileNetV1 (Frozen) Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_mobilenet_frozen.history['loss'], label='Train Loss')
plt.plot(history_mobilenet_frozen.history['val_loss'], label='Val Loss')
plt.title("MobileNetV1 (Frozen) Loss")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm_mobilenet_frozen = confusion_matrix(y_true_mobilenet, y_pred_mobilenet_frozen)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_mobilenet_frozen, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title("Confusion Matrix (MobileNetV1 Frozen)")
plt.show()


In [ ]:
x_test_batch, y_test_batch = next(test_gen_mobilenet)
y_pred_batch_mobilenet = np.argmax(mobilenet_frozen_model.predict(x_test_batch), axis=1)

random_indices = np.random.permutation(x_test_batch.shape[0])[:9]
plt.figure(figsize=(10, 10))
for i, idx in enumerate(random_indices):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test_batch[idx])
    pred_label = class_names[y_pred_batch_mobilenet[idx]]
    true_label = class_names[int(y_test_batch[idx])]
    plt.title(f"P: {pred_label}\nT: {true_label}", fontsize=10, color='white', backgroundcolor='black', loc='left')
    plt.axis('off')
plt.suptitle("Sample Predictions (MobileNetV1 Frozen)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
# Unlock some layers for fine-tuning
base_model = mobilenet_frozen_model.layers[1]  # Extract the base model from frozen model
base_model.trainable = True

# Fine-tune the last few layers (e.g., last 4 layers)
fine_tune_at = len(base_model.layers) - 4
for layer in base_model.layers[-8:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True


In [ ]:
mobilenet_frozen_model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
mobilenet_finetune_epochs = 50
start_time = time.time()

history_mobilenet_finetuned = mobilenet_frozen_model.fit(
    train_gen_mobilenet,
    validation_data=val_gen_mobilenet,
    epochs=mobilenet_finetune_epochs,
    callbacks=mobilenet_callbacks
)

mobilenet_finetuned_time = time.time() - start_time


In [ ]:
test_loss_mobilenet_finetuned, test_acc_mobilenet_finetuned = mobilenet_frozen_model.evaluate(test_gen_mobilenet, verbose=0)
print(f"MobileNetV1 (Fine-Tuned) Test Accuracy: {test_acc_mobilenet_finetuned:.4f}")

# Get predictions
y_pred_mobilenet_finetuned = np.argmax(mobilenet_frozen_model.predict(test_gen_mobilenet), axis=1)
y_true_mobilenet_finetuned = test_gen_mobilenet.classes


In [ ]:
print("Classification Report (MobileNetV1 Fine-Tuned):")
print(classification_report(y_true_mobilenet_finetuned, y_pred_mobilenet_finetuned, target_names=class_names))


In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_mobilenet_finetuned.history['accuracy'], label='Train Acc')
plt.plot(history_mobilenet_finetuned.history['val_accuracy'], label='Val Acc')
plt.title("MobileNetV1 (Fine-Tuned) Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_mobilenet_finetuned.history['loss'], label='Train Loss')
plt.plot(history_mobilenet_finetuned.history['val_loss'], label='Val Loss')
plt.title("MobileNetV1 (Fine-Tuned) Loss")
plt.legend()
plt.show()


In [ ]:
cm_mobilenet_finetuned = confusion_matrix(y_true_mobilenet_finetuned, y_pred_mobilenet_finetuned)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_mobilenet_finetuned, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title("Confusion Matrix (MobileNetV1 Fine-Tuned)")
plt.show()


In [ ]:
# Get a batch of test data
x_test_batch, y_test_batch = next(test_gen_mobilenet)

# Predict using the fine-tuned model
y_pred_batch_mobilenet_finetuned = np.argmax(mobilenet_frozen_model.predict(x_test_batch), axis=1)

# Select random indices for displaying predictions
random_indices = np.random.permutation(x_test_batch.shape[0])[:9]

# Plotting the images and their predictions
plt.figure(figsize=(10, 10))
for i, idx in enumerate(random_indices):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_test_batch[idx])  # Display the image
    pred_label = class_names[y_pred_batch_mobilenet_finetuned[idx]]  # Predicted label
    true_label = class_names[int(y_test_batch[idx])]  # True label
    plt.title(f"P: {pred_label}\nT: {true_label}", fontsize=10, color='white', backgroundcolor='black', loc='left')
    plt.axis('off')

# Set the overall title for the plot
plt.suptitle("Sample Predictions (MobileNetV1 Fine-Tuned)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to avoid overlap
plt.show()


In [ ]:
# Print all test accuracies
print(f"{'Model':<30} {'Optimizer':<10} {'Test Accuracy':<15}")
print("-" * 55)
# Baseline model
print(f"{'Baseline CNN':<30} {'Adam':<10} {test_acc:.4f}")

# Deeper CNN with Adam
print(f"{'Deeper CNN':<30} {'Adam':<10} {adam_acc:.4f}")

# Deeper CNN with SGD
print(f"{'Deeper CNN':<30} {'SGD':<10} {sgd_acc:.4f}")

# MobileNetV1 (Frozen)
print(f"{'MobileNetV1 (Frozen)':<30} {'Adam':<10} {test_acc_mobilenet_frozen:.4f}")

# MobileNetV1 (Fine-Tuned)
print(f"{'MobileNetV1 (Fine-Tuned)':<30} {'Adam':<10} {test_acc_mobilenet_finetuned:.4f}")
